In [1]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

# -1) Cycliste

In [2]:
class logger():
    """
    Save data into memory (in an array) and write it down to a file after [nb_log] is reached.
    """
    def __init__(self, path, limit_to_save_to_disk=100):
        self.path    = path
        self.limit   = limit_to_save_to_disk
        self.logs    = []
        self.nb_logs = 0
        
    def log(self, string):
        self.logs.append(string)
        self.nb_logs = self.nb_logs +1
        if self.limit < self.nb_logs:
            self.write_to_disk()
    
    def write_to_disk(self):
        with open(self.path, "a") as f:
            while len(self.logs):
                try:
                    a = self.logs.pop()
                    f.write(u"%s\n"%a.decode('ascii', errors='ignore'))
                except:
                    print "pbm with "
                    print a
        self.nb_logs = len(self.logs)
    def __del__(self):
        self.write_to_disk()
        

In [3]:
class Cycliste:
        
    def __init__(self, identifiant):
        import time
        from scipy.stats import norm
        import random
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        if age < 35:
            facteur = norm( loc =1.2, scale=0.2 )
        elif age <50:
            facteur = norm( loc =1  , scale=0.2 )
        elif age <90:
            facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base = 20
            nb_km = norm( loc =7, scale=2 )
        else:
            vitesse_de_base = 15
            nb_km = norm( loc =10, scale=2 )
        vitesse_moyenne = vitesse_de_base * facteur.rvs(1) + sportif
        
        self.sportif     = sportif
        self.age         = age
        self.sexe        = sexe
        self.nb_km       = nb_km.rvs(1)[0]
        self.vitesse     = vitesse_moyenne[0]
        self.sur_velo    = False
        self.debug       = False
        self.nb_trajet   = 0
        self.durees      = []
        self.trajets     = []
        self.velo        = False
        self.identifiant = "cycliste_%s"%identifiant
        self.prises      = []
        self.attente         = random.randint(0,100)
        self.last_rendu      = time.time()
        self.logger_cycliste = logger("./logs/cycliste_cyclistes.csv")
        self.logger_prise    = logger("./logs/cycliste_prises.csv")
        self.logger_rendu    = logger("./logs/cycliste_rendu.csv")
        self.logger_debug    = logger("./logs/cycliste_debug.csv")
        
        self.log_cycliste()
        
    def log_debug(self, message):
        from time import time
        to_print = u"%s,%s,%s"% (self.identifiant, time() , message)
        self.logger_debug.log(to_print)
    
    def peut_prendre_velo(self, station):
        """
        Renvoie un velo ou bien False.
        """
        from time import time
        if time() < self.attente + self.last_rendu:
            self.log_debug(u"trop tot")
            return False
        if self.sur_velo:
            self.log_debug(u"deja sur velo")
            return False
        velo = station.prendre_velo()
        if not velo:
            self.log_debug(u"pas de velo sur la station %s " % station)
            return False
        return velo
        
    def prend_velo(self, station):
        import time
        
        velo = self.peut_prendre_velo(station)
        if not velo :
            return
        
        velo.disponible      = False
        self.sur_velo        = True
        self.heure_de_depart = time.time()
        self.velo            = velo
        self.log_prise()
        try:
            nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
        except Exception as e:
            self.log_debug(u"pbm sur nb_km_local = self.nb_km = %s " %self.nb_km)
            nb_km_local = 15
        try:
            vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
            vitesse_local        = vitesse_local * velo.performance
        except:
            self.log_debug(u"pbm sur vitesse_local self.vitesse = %s"%self.vitesse)
            vitesse_local = self.vitesse
            
        self.nb_km_trajet    = nb_km_local
        duree                = nb_km_local / vitesse_local
        self.heure_d_arrivee = self.heure_de_depart + duree
        
        if velo.performance <0.1:
            duree                = 0
            self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart
            self.nb_km_trajet    = 0
            self.rendre_velo(station)
        elif velo.performance <0.3:
            duree                = duree / 10
            self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart + duree
            self.nb_km_trajet    = self.nb_km_trajet / float(10)
            self.rendre_velo(station)
        elif velo.performance <0.5:
            duree                = duree / 5
            self.durees.append( duree)
            self.nb_km_trajet    = self.nb_km_trajet / float(5)
            self.heure_d_arrivee =  self.heure_de_depart + duree 
        else:
            self.durees.append( duree)




    def rendre_velo(self, station):        
        import time
        if not self.sur_velo:
            if self.debug : print u"pas en chemin",
            return 0
        if time.time() > self.heure_d_arrivee:
            if station.rendre_velo(self.velo, self.nb_km_trajet):
                
                #if self.debug : print "plus sur velo",
                duree_constatee = self.heure_d_arrivee - self.heure_de_depart
                trajet          = (self.heure_de_depart, self.heure_d_arrivee, self.identifiant, self.nb_km_trajet, self.velo.nom )
                self.log_rendu()
                self.last_rendu = time.time()
                self.sur_velo  = False
                self.nb_trajet = self.nb_trajet + 1
                self.velo      = False
                self.trajets.append(trajet)
                
            else:
                if self.debug : print u"impossible de rendre sur station %s"%(station),
                pass
        else:
            if self.debug : print u"pas encore arrive",
            pass
        
    def log_cycliste(self):
        to_print = u"%s,%s,%s,%s,%s,%s,%s"%(    self.identifiant   ,self.sportif     ,
                                                 self.age         ,self.sexe        ,
                                                 self.nb_km       ,self.vitesse     , self.attente )
        
        self.logger_cycliste.log(to_print)
            
    def log_prise(self, action="prise"):
        to_print = u"%s,%s,%s,%s"%( self.identifiant,self.heure_de_depart, self.velo.nom, action)
        self.logger_prise.log(to_print)
            
    def log_rendu(self, action="prise"):
        to_print = u"%s,%s,%s,%s,%s\n"%( self.identifiant, self.heure_de_depart, 
                                       self.heure_d_arrivee,  self.nb_km_trajet, self.velo.nom )

        self.logger_rendu.log(to_print)


# -1) Velo

In [4]:
class Velo:

    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        self.nom          = nom
        performance       = norm( loc =0.9, scale=0.2 )
        self.performance  = performance.rvs(1)[0]
        self.station      = station
        self.degradation  = 0
        self.performances = []
        self.debug        = False
        self.nb_km_trajet = 0
        self.heures_rendu = []
        self.disponible   = True
        self.logger_reparation = logger("./logs/velo_reparations.csv")
        self.logger_etat       = logger("./logs/velos_etats.csv")
        self.probleme_list     = self.get_problemes_list()

    def get_problemes_list(self):
        """
        Créé le tableau de dégradations possible pour un vélo.
        """
        problemes       = [0] * 50
        problemes_bis   = {u"pedale" : 0.2, u"roue"  : 1 , u"degonfle" : 0.3, u"freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.values())
        return problemes
        
    def log_reparation(self):
        import time
        reparation =(self.nom,time.time(),self.performance)
        to_print = u"%s,%s,%s"%reparation
        self.logger_reparation.log(to_print)
        

    def log_etat(self):
        from time import time
        to_print = u"%s,%s,%s,%s,%s\n"%(   self.nom           ,
                                           time()            ,  
                                           self.station.nom  ,
                                           self.performance  ,
                                           self.nb_km_trajet )
        self.logger_etat.log(to_print)
    
    def rendu(self, station, nb_km_trajet):
        import time
        self.disponible = True
        if self.debug : print(u"velo %s rendu sur station %s"%(self, station))
            
        degradation       = random.choice(self.probleme_list)        
        self.performance  = self.performance - self.performance*degradation                                        
        self.nb_km_trajet = self.nb_km_trajet + nb_km_trajet
        self.station = station
        self.log_etat()
        if 900 < random.randint(0, 1000 ) :
            self.performance  = 1
            self.log_reparation()
        
        

    def __str__(self):
        return "%s"%self.nom



# -1 ) Station

In [5]:
class Station:
    def __init__(self, nom, debug=False ):
        import random
        import time
        debut         = time.time()
        self.nom      = nom
        self.nb_plot  = random.choice([10,15,20, 30])
        self.nb_libre = 0 #random.randint(0, self.nb_plot )
        self.nb_velos = self.nb_plot # - self.nb_libre
        self.velos    = []
        noms          = get_names()
        
        for n in range(self.nb_velos ):
            nom_velo= u"velo_"+"".join(["%s"%x for x in random.choice(noms)])
            self.velos.append(Velo(nom_velo , self))
        
        if debug : print u"created %s velos"%len(self.velos),
        self.debug          = False
        self.avaries        = []
        self.arrete         = False
        duree               = time.time() - debut
        if debug : print u"station créée en %s"%duree
        
        self.logger_dispos = logger("./logs/stations_dispos.csv")
        self.logger_reparation = logger("./logs/stations_reparations.csv")
        self.logger_avarie = logger("./logs/stations_avarie.csv")
        
        self.liste_d_avaries = self.get_avarie_list()
        self.log_dispo()


    def log_reparation(self):
        from time import time
        dispo =(self.nom, time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s"%dispo
        self.logger_reparation.log(to_print)
            
            
    def log_dispo(self):
        from time import time
        dispo =(self.nom,time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s"%dispo
        self.logger_dispos.log(to_print)
            
    def log_avarie(self, avarie):
        from time import time
        to_print = u"%s,%s,%s"%(self.nom, time(), avarie )
        self.logger_avarie.log(to_print)

        
    def get_avarie_list(self):
        avaries     = [0]*100000
        plot        = u"plot mort"
        half        = u"50% mort"
        network     = u"reseau"
        electricite = u"eteint"
        bug         = u"bug"
        problemes   = { plot       : 20 ,
                       half        : 10 ,
                       network     :  5 ,
                       electricite : 30 , 
                       bug         :  3 }
        
        for k,v in problemes.iteritems():
            avaries.extend([k] * v)
        return avaries
    
    def avarie_possible(self):
        import math
        import random
        import time
        
        plot        = u"plot mort"
        half        = u"50% mort"
        network     = u"reseau"
        electricite = u"eteint"
        bug         = u"bug"
        
        avaries = list(self.liste_d_avaries)
        if plot in self.avaries : avaries.extend( [plot] * 60 )
        if half in self.avaries : avaries.extend( [half] *100 )
        
        avarie = random.choice(avaries)
        self.avaries.extend([avarie])
        if avarie == plot :
            self.nb_libre = self.nb_libre -1
        elif avarie == half:
            self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
        elif avarie == network :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == electricite :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == bug :
            if self.nb_libre >0:
                self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
        if self.nb_libre <0:
            self.nb_libre = 0            
        self.log_dispo()
        
        if avarie:
            self.log_avarie("%s"%avarie)
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.log_dispo()
            return 1
        if 9500 < random.randint(0, 10000 ) :
            self.nb_libre = self.nb_plot - self.nb_velos
            self.log_reparation()
            
        return 0

    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.log_dispo()
            return to_return

        return 0
    def __str__(self):
        return u"station %s : self.nb_libre = %s, self.nb_velos =%s "%(self.nom,self.nb_libre, self.nb_velos)


# 0) get_names

In [6]:
def get_names(nb_lettres=5, nb_noms = 15000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        nb_noms = nb_noms -1
        if nb_noms <2:
            break
    shuffle(names)
    return names

# 0) création des stations

In [7]:
from time import time
import multiprocessing
n = 4
debut = time()
process_pool = multiprocessing.Pool(n)
debut        = time()
names        = get_names()
noms         = [names[i] for i in range(50)]
stations     = process_pool.map(Station, noms)
nb_velos     = sum([len(station.velos) for station in stations])
duree = time() - debut
print "duree = %s pour %s cree soit %s sec / unite"%(duree, nb_velos, duree/nb_velos)


duree = 0.431797981262 pour 945 cree soit 0.000456929080701 sec / unite


# Creation des cycliste

In [8]:
n = 4
process_pool = multiprocessing.Pool(n)
names        = get_names(5, nb_velos*1.2)
noms         = [names[i] for i in range(nb_velos)]
del names
cyclistes   = process_pool.map(Cycliste, noms)
print "duree = %s pour %s cree soit %s sec / unite"%(duree, len(cyclistes), duree/nb_velos)


duree = 0.431797981262 pour 945 cree soit 0.000456929080701 sec / unite


# voyages

In [9]:
from time import time
import multiprocessing
import math
from time import time

In [10]:
!rm ./logs/*

In [11]:
depart = time()
n = 0  
while time() < depart + 3600 : 
    a         = time()
    old_nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
    en_route = filter(lambda cycliste:     cycliste.sur_velo, cyclistes)
    a_pied   = filter(lambda cycliste: not cycliste.sur_velo, cyclistes)
    map(lambda cycliste: cycliste.rendre_velo(random.choice(stations)), en_route  )
    map(lambda cycliste: cycliste.prend_velo(random.choice(stations)),  a_pied    )
    new_nb_trajet =  sum([cycliste.nb_trajet for cycliste in cyclistes]) - old_nb_trajet
    duree_totale = time()-a
    print "(%03d)%03d/% 3.2f|"%(n, new_nb_trajet, duree_totale),

    n += 1
print "*"*30
print "fini"
print "*"*30

(000)000/ 0.06| (001)000/ 0.01| (002)000/ 0.01| (003)000/ 0.01| (004)000/ 0.01| (005)000/ 0.01| (006)000/ 0.01| (007)000/ 0.01| (008)000/ 0.01| (009)000/ 0.01| (010)000/ 0.01| (011)000/ 0.01| (012)000/ 0.01| (013)000/ 0.01| (014)000/ 0.01| (015)000/ 0.01| (016)000/ 0.01| (017)000/ 0.01| (018)000/ 0.01| (019)001/ 0.01| (020)001/ 0.01| (021)000/ 0.01| (022)000/ 0.01| (023)001/ 0.01| (024)000/ 0.01| (025)000/ 0.01| (026)000/ 0.01| (027)001/ 0.01| (028)000/ 0.01| (029)002/ 0.01| (030)000/ 0.02| (031)001/ 0.01| (032)000/ 0.01| (033)000/ 0.01| (034)000/ 0.01| (035)001/ 0.01| (036)000/ 0.03| (037)001/ 0.02| (038)000/ 0.01| (039)001/ 0.01| (040)001/ 0.02| (041)000/ 0.01| (042)001/ 0.01| (043)000/ 0.01| (044)000/ 0.01| (045)000/ 0.01| (046)000/ 0.01| (047)000/ 0.01| (048)000/ 0.02| (049)000/ 0.02| (050)000/ 0.02| (051)002/ 0.02| (052)000/ 0.02| (053)001/ 0.02| (054)001/ 0.01| (055)000/ 0.01| (056)000/ 0.01| (057)000/ 0.01| (058)000/ 0.01| (059)000/ 0.01| (060)000/ 0.01| (061)000/ 0.02| (062)001

/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:101: RuntimeWarning: divide by zero encountered in double_scalars


 (111)001/ 0.01| (112)000/ 0.02| (113)000/ 0.01| (114)000/ 0.01| (115)002/ 0.01| (116)001/ 0.02| (117)000/ 0.01| (118)000/ 0.01| (119)000/ 0.01| (120)000/ 0.02| (121)003/ 0.01| (122)000/ 0.01| (123)001/ 0.01| (124)001/ 0.02| (125)000/ 0.02| (126)000/ 0.01| (127)000/ 0.01| (128)000/ 0.01| (129)000/ 0.01| (130)000/ 0.01| (131)000/ 0.02| (132)000/ 0.02| (133)002/ 0.01| (134)000/ 0.01| (135)000/ 0.01| (136)000/ 0.01| (137)000/ 0.02| (138)003/ 0.02| (139)000/ 0.01| (140)001/ 0.02| (141)001/ 0.01| (142)000/ 0.01| (143)000/ 0.01| (144)000/ 0.01| (145)000/ 0.01| (146)001/ 0.01| (147)000/ 0.01| (148)000/ 0.01| (149)000/ 0.01| (150)000/ 0.01| (151)000/ 0.02| (152)002/ 0.01| (153)000/ 0.01| (154)000/ 0.01| (155)000/ 0.02| (156)003/ 0.02| (157)000/ 0.02| (158)001/ 0.02| (159)001/ 0.02| (160)000/ 0.02| (161)001/ 0.01| (162)001/ 0.01| (163)000/ 0.02| (164)000/ 0.01| (165)000/ 0.01| (166)000/ 0.01| (167)001/ 0.01| (168)000/ 0.01| (169)001/ 0.02| (170)000/ 0.02| (171)000/ 0.01| (172)000/ 0.02| (173)00

KeyboardInterrupt: 